In [1]:
import pandas as pd
import numpy as np

In [2]:
divar_df = pd.read_csv('Divar.csv')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2008\2646132604.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  divar_df = pd.read_csv('Divar.csv')


In [3]:
divar_df = divar_df.drop(['description', 'title', 'rent_to_single', 'rooms_count', 'has_balcony',], axis=1)




In [3]:

divar_df = divar_df.drop(['has_barbecue'], axis=1)


In [4]:
divar_df = divar_df.drop(['has_restroom'], axis=1)

In [9]:
print(divar_df['has_heating_system'].value_counts())

has_heating_system
0    1000000
Name: count, dtype: int64


In [6]:
target_heating_syste = ['shoofaj', 'heater', 'duct_split', 'floor_heating', 'split', 'fan_coil' , 'fireplace','unselect' ]

divar_df['has_heating_system'] = divar_df['has_heating_system'].isin(target_heating_syste).astype(int)

In [7]:
print(divar_df['has_heating_system'].value_counts())

has_heating_system
0    1000000
Name: count, dtype: int64


In [10]:
print(divar_df['has_cooling_system'].value_counts())

has_cooling_system
water_cooler       219266
air_conditioner     52341
split               38708
duct_split          29373
unselect             5917
fan_coil             5014
Name: count, dtype: int64


In [11]:
target_cooling_system = ['water_cooler', 'air_conditioner', 'split', 'duct_split', 'unselect']

divar_df['has_cooling_system'] = divar_df['has_cooling_system'].isin(target_cooling_system).astype(int)

In [12]:
print(divar_df['has_cooling_system'].value_counts())

has_cooling_system
0    654395
1    345605
Name: count, dtype: int64


In [13]:
print(divar_df['has_warm_water_provider'].value_counts())

has_warm_water_provider
package         205641
water_heater    141082
powerhouse       31395
unselect          1382
Name: count, dtype: int64


In [14]:
target_warm_water_provider = ['package', 'water_heater', 'powerhouse', 'unselect']

divar_df['has_warm_water_provider'] = divar_df['has_warm_water_provider'].isin(target_warm_water_provider).astype(int)

In [15]:
print(divar_df['has_warm_water_provider'].value_counts())

has_warm_water_provider
0    620500
1    379500
Name: count, dtype: int64


In [16]:
divar_df['has_system_columns'] = divar_df['has_cooling_system'] + divar_df['has_heating_system'] + divar_df['has_warm_water_provider']

In [17]:
print(divar_df['has_system_columns'].value_counts())

has_system_columns
0    603723
2    328828
1     67449
Name: count, dtype: int64


In [18]:
print(divar_df['has_security_guard'].value_counts())

has_security_guard
False    24631
True      6681
Name: count, dtype: int64


In [19]:
luxury_columns = ['has_pool', 'has_sauna', 'has_jacuzzi', 'has_security_guard']

divar_df['has_luxury_features'] = divar_df[luxury_columns].any(axis=1)

divar_df = divar_df.drop(columns=luxury_columns)

In [20]:
print(divar_df['has_luxury_features'].value_counts())

has_luxury_features
False    991767
True       8233
Name: count, dtype: int64


In [21]:
infra_cols = ['has_water', 'has_electricity', 'has_gas']
divar_df['infrastructure_score'] = divar_df[infra_cols].sum(axis=1)
divar_df = divar_df.drop(columns=infra_cols)

In [18]:
divar_df.shape
divar_df.info

<bound method DataFrame.info of         Unnamed: 0         cat2_slug         cat3_slug      city_slug  \
0                0    temporary-rent             villa          karaj   
1                1  residential-sell    apartment-sell         tehran   
2                2  residential-rent    apartment-rent         tehran   
3                3   commercial-rent       office-rent         tehran   
4                4  residential-sell    apartment-sell        mashhad   
...            ...               ...               ...            ...   
999995      999995  residential-sell    apartment-sell     kermanshah   
999996      999996  residential-rent    apartment-rent         tehran   
999997      999997  residential-sell  house-villa-sell           yazd   
999998      999998    temporary-rent   suite-apartment  bandar-anzali   
999999      999999  residential-rent    apartment-rent         tehran   

       neighborhood_slug     created_at_month    user_type rent_mode  \
0              mehr

In [22]:
facility_score = ['has_parking', 'has_elevator', 'has_warehouse']
divar_df['facility_score'] = divar_df[facility_score].sum(axis=1)
divar_df = divar_df.drop(columns=facility_score)

In [23]:
divar_df['total_floors_count'] = pd.to_numeric(divar_df['total_floors_count'], errors='coerce')

In [24]:
nan_counts = divar_df['total_floors_count'].isna().sum()


print(nan_counts)

696006


In [25]:
divar_df['total_floors_count'] = divar_df.groupby(['city_slug', 'neighborhood_slug'])['total_floors_count'].transform(
    lambda x: x.fillna(x.mean())
)

In [26]:
mask = divar_df['total_floors_count'].isna()

divar_df.loc[mask, 'total_floors_count'] = divar_df.groupby('city_slug')['total_floors_count'].transform(
    lambda x: x.fillna(x.mean())
)[mask]

In [27]:
divar_df['total_floors_count'].fillna(divar_df['total_floors_count'].mean(), inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2008\3948902769.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_df['total_floors_count'].fillna(divar_df['total_floors_count'].mean(), inplace=True)


In [28]:
divar_df['total_floors_count'] = divar_df['total_floors_count'].round().astype('Int64')

In [29]:
divar_df['unit_per_floor'] = pd.to_numeric(divar_df['unit_per_floor'], errors='coerce')

In [30]:
divar_df['unit_per_floor'] = divar_df.groupby(['city_slug', 'neighborhood_slug'])['unit_per_floor'].transform(
    lambda x: x.fillna(x.mean())
)

In [31]:
mask = divar_df['unit_per_floor'].isna()

divar_df.loc[mask, 'unit_per_floor'] = divar_df.groupby('city_slug')['unit_per_floor'].transform(
    lambda x: x.fillna(x.mean())
)[mask]

In [32]:
divar_df['unit_per_floor'].fillna(divar_df['unit_per_floor'].mean(), inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2008\1761935966.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  divar_df['unit_per_floor'].fillna(divar_df['unit_per_floor'].mean(), inplace=True)


In [33]:
divar_df['unit_per_floor'] = divar_df['unit_per_floor'].round().astype('Int64')

In [34]:
divar_df['total_units'] = divar_df['total_floors_count'] * divar_df['unit_per_floor']
divar_df['density_per_floor'] = divar_df['unit_per_floor'] / divar_df['total_units']


In [35]:
divar_df = divar_df.drop(columns=['total_units', 'unit_per_floor'])

In [36]:
print(divar_df['building_direction'].value_counts())

building_direction
south       159453
north       142009
east         12111
west          7143
unselect      3207
Name: count, dtype: int64


In [37]:
divar_df['building_direction'] = divar_df['building_direction'].fillna('نامشخص')
divar_df.loc[divar_df['building_direction'] == 'unselect', 'building_direction'] = 'نامشخص'


In [38]:

divar_df = pd.get_dummies(divar_df, columns=['building_direction'], prefix='dir')

In [39]:
divar_df.shape
divar_df.info

<bound method DataFrame.info of         Unnamed: 0         cat2_slug         cat3_slug      city_slug  \
0                0    temporary-rent             villa          karaj   
1                1  residential-sell    apartment-sell         tehran   
2                2  residential-rent    apartment-rent         tehran   
3                3   commercial-rent       office-rent         tehran   
4                4  residential-sell    apartment-sell        mashhad   
...            ...               ...               ...            ...   
999995      999995  residential-sell    apartment-sell     kermanshah   
999996      999996  residential-rent    apartment-rent         tehran   
999997      999997  residential-sell  house-villa-sell           yazd   
999998      999998    temporary-rent   suite-apartment  bandar-anzali   
999999      999999  residential-rent    apartment-rent         tehran   

       neighborhood_slug     created_at_month    user_type  \
0              mehrshahr  202

In [40]:
divar_df['ad_year'] = pd.to_datetime(divar_df['created_at_month']).dt.year

In [41]:
print(divar_df['ad_year'].value_counts())

ad_year
2024    996946
2023      1689
2025      1292
2022        65
2021         6
2020         2
Name: count, dtype: int64


In [42]:
def convert_persian_digits(text):
    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    english_digits = '0123456789'
    translation_table = str.maketrans(persian_digits, english_digits)
    return str(text).translate(translation_table)

In [43]:
divar_df['construction_year'] = divar_df['construction_year'].apply(convert_persian_digits)


divar_df['construction_year'] = pd.to_numeric(divar_df['construction_year'], errors='coerce')

In [44]:
print(divar_df['construction_year'].value_counts())

construction_year
1403.0    116260
1390.0     59139
1402.0     58424
1400.0     53674
1395.0     53029
1398.0     38207
1397.0     36326
1396.0     35487
1401.0     35328
1385.0     34065
1399.0     29594
1393.0     29094
1392.0     26130
1394.0     26110
1388.0     24268
1380.0     23480
1389.0     16755
1391.0     16316
1387.0     14136
1386.0     13468
1383.0      9894
1384.0      8494
1375.0      7247
1382.0      6965
1371.0      5531
1381.0      3590
1378.0      3025
1379.0      2415
1377.0      2117
1372.0      1914
1373.0      1827
1376.0      1593
1374.0      1289
Name: count, dtype: int64


In [45]:
divar_df['ad_year'] = divar_df['ad_year'].round().astype('Int32')

In [46]:
divar_df['building_age'] = divar_df['ad_year'] - divar_df['construction_year'] - 621
divar_df['building_age'] = divar_df['building_age'].clip(lower=0)  

In [47]:
divar_df = divar_df.drop('construction_year', axis=1)

In [48]:
print(divar_df['building_age'].value_counts())

building_age
0.0     116310
13.0     59091
1.0      58447
3.0      53558
8.0      52991
5.0      38204
6.0      36311
7.0      35510
2.0      35309
18.0     34039
4.0      29629
10.0     29074
11.0     26152
9.0      26150
15.0     24255
23.0     23449
14.0     16787
12.0     16342
16.0     14147
17.0     13479
20.0      9887
19.0      8516
28.0      7239
21.0      6969
32.0      5523
22.0      3615
25.0      3025
24.0      2427
26.0      2116
31.0      1918
30.0      1825
27.0      1599
29.0      1295
33.0         3
Name: count, dtype: Int64


In [49]:
divar_df['effective_age'] = divar_df['building_age'] - (divar_df['is_rebuilt'] * 10)
divar_df['effective_age'] = divar_df['effective_age'].clip(lower=0) 

In [50]:
print(divar_df['effective_age'].value_counts())

effective_age
0       142685
3.0      40755
8.0      34587
1.0      34065
13.0     30345
5.0      25381
2.0      21677
6.0      21170
7.0      20492
4.0      19270
10.0     16452
9.0      14635
18.0     14101
11.0     13947
15.0      9921
23.0      8370
12.0      8269
14.0      7421
16.0      5743
17.0      5287
20.0      3715
19.0      3263
21.0      2924
28.0      2794
22.0      2623
32.0      2220
25.0      1089
24.0       933
26.0       817
31.0       770
30.0       719
27.0       614
29.0       498
33.0         2
Name: count, dtype: int64


In [51]:
divar_df['is_rebuilt'] = divar_df['is_rebuilt'].fillna(False).astype(int)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2008\476863760.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  divar_df['is_rebuilt'] = divar_df['is_rebuilt'].fillna(False).astype(int)


In [52]:
divar_df = divar_df.drop('is_rebuilt', axis=1)

In [53]:
print(divar_df['floor_material'].value_counts())

floor_material
ceramic             307821
stone                38395
carpet               17696
mosaic               14611
wood_parquet         14419
laminate_parquet      9582
floor_covering        1793
unselect              1667
Name: count, dtype: int64


In [54]:
material_rank = {
    'carpet': 0,
    'mosaic': 1,
    'ceramic': 2,
    'stone': 3,
    'laminate_parquet': 4,
    'wood_parquet': 5
}

In [55]:
divar_df['floor_material'] = divar_df['floor_material'].replace('floor_covering', np.nan)
divar_df['floor_material'] = divar_df['floor_material'].replace('unselect', np.nan)

In [56]:
divar_df['floor_material_rank'] = divar_df['floor_material'].map(material_rank)

In [57]:
divar_df['floor_material_rank'] = divar_df['floor_material_rank'].fillna(0).astype(int)

In [58]:
print(divar_df['floor_material_rank'].value_counts())

floor_material_rank
0    615172
2    307821
3     38395
1     14611
5     14419
4      9582
Name: count, dtype: int64


In [59]:
divar_df = divar_df.drop(['location_latitude', 'location_longitude', 'location_radius'], axis=1)


In [60]:
divar_df = divar_df[~divar_df['cat3_slug'].isin(['other', 'partnership'])]

In [61]:
divar_df = divar_df[~divar_df['cat2_slug'].isin(['real-estate-services'])]


In [62]:
divar_df.shape
divar_df.info

<bound method DataFrame.info of         Unnamed: 0         cat2_slug         cat3_slug      city_slug  \
0                0    temporary-rent             villa          karaj   
1                1  residential-sell    apartment-sell         tehran   
2                2  residential-rent    apartment-rent         tehran   
3                3   commercial-rent       office-rent         tehran   
4                4  residential-sell    apartment-sell        mashhad   
...            ...               ...               ...            ...   
999995      999995  residential-sell    apartment-sell     kermanshah   
999996      999996  residential-rent    apartment-rent         tehran   
999997      999997  residential-sell  house-villa-sell           yazd   
999998      999998    temporary-rent   suite-apartment  bandar-anzali   
999999      999999  residential-rent    apartment-rent         tehran   

       neighborhood_slug     created_at_month    user_type  \
0              mehrshahr  202

In [63]:

def get_property_type(cat2_slug):
    if pd.isna(cat2_slug):
        return np.nan
    elif cat2_slug.startswith('residential'):
        return int(1)
    elif cat2_slug.startswith('commercial'):
        return int(2)
    else:
        return np.nan

divar_df['deal_type'] = divar_df['cat2_slug'].apply(get_property_type)

In [64]:
print(divar_df['deal_type'].value_counts())

deal_type
1.0    835266
2.0    115428
Name: count, dtype: int64


In [66]:
df = pd.read_csv('Divar.csv')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2008\1116618770.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Divar.csv')


In [67]:
divar_df['rent/sell'] = df['cat2_slug']

In [68]:
def get_rent_or_sell(cat2_slug):
    if pd.isna(cat2_slug):
        return np.nan
    elif 'rent' in cat2_slug:
        return 0
    elif 'sell' in cat2_slug:
        return 1
    else:
        return np.nan

divar_df['rent/sell'] = divar_df['rent/sell'].apply(get_rent_or_sell)

In [69]:
print(divar_df['rent/sell'].value_counts())

rent/sell
1    597569
0    383028
Name: count, dtype: int64


In [70]:
divar_df = divar_df.drop(['cat2_slug', 'cat3_slug'], axis=1)

In [71]:
sell_df = divar_df[divar_df['rent/sell'] == '1'].copy()
sell_df['target'] = sell_df['price_value']

# فقط ردیف‌های اجاره:
rent_df = divar_df[divar_df['rent/sell'] == '0'].copy()
rent_df['target'] = rent_df['rent_value'] + (rent_df['credit_value'] / 4)

In [72]:
print(rent_df[rent_df['rent_mode'] == 'مقطوع'])

Empty DataFrame
Columns: [Unnamed: 0, city_slug, neighborhood_slug, created_at_month, user_type, description, title, rent_mode, rent_value, rent_to_single, rent_type, price_mode, price_value, credit_mode, credit_value, rent_credit_transform, transformable_price, transformable_credit, transformed_credit, transformable_rent, transformed_rent, land_size, building_size, deed_type, has_business_deed, floor, rooms_count, total_floors_count, has_balcony, has_warm_water_provider, has_heating_system, has_cooling_system, floor_material, property_type, regular_person_capacity, extra_person_capacity, cost_per_extra_person, rent_price_on_regular_days, rent_price_on_special_days, rent_price_at_weekends, has_system_columns, has_luxury_features, infrastructure_score, facility_score, density_per_floor, dir_east, dir_north, dir_south, dir_west, dir_نامشخص, ad_year, building_age, effective_age, floor_material_rank, deal_type, rent/sell, target]
Index: []

[0 rows x 57 columns]


In [61]:
del df

شروع مدل اجاره







In [ ]:
rent_df = rent_df.drop(['cost_per_extra_person', 'cat3_slug'], axis=1)

شروع مدل فروش





In [73]:



sell_df = sell_df.drop(['rent_mode', 'rent_value', 'rent_type', 'credit_mode', 'credit_value', 'rent_credit_transform', 'transformable_price', 'transformable_credit', 'transformed_credit', 'transformable_rent','transformed_rent','regular_person_capacity', 'extra_person_capacity', 'rent_price_on_regular_days', 'rent_price_on_special_days', 'rent_price_at_weekends'], axis=1)



In [74]:
sell_df.shape
sell_df.info

<bound method DataFrame.info of Empty DataFrame
Columns: [Unnamed: 0, city_slug, neighborhood_slug, created_at_month, user_type, description, title, rent_to_single, price_mode, price_value, land_size, building_size, deed_type, has_business_deed, floor, rooms_count, total_floors_count, has_balcony, has_warm_water_provider, has_heating_system, has_cooling_system, floor_material, property_type, cost_per_extra_person, has_system_columns, has_luxury_features, infrastructure_score, facility_score, density_per_floor, dir_east, dir_north, dir_south, dir_west, dir_نامشخص, ad_year, building_age, effective_age, floor_material_rank, deal_type, rent/sell, target]
Index: []

[0 rows x 41 columns]>

In [75]:
print(sell_df.columns)


Index(['Unnamed: 0', 'city_slug', 'neighborhood_slug', 'created_at_month',
       'user_type', 'description', 'title', 'rent_to_single', 'price_mode',
       'price_value', 'land_size', 'building_size', 'deed_type',
       'has_business_deed', 'floor', 'rooms_count', 'total_floors_count',
       'has_balcony', 'has_warm_water_provider', 'has_heating_system',
       'has_cooling_system', 'floor_material', 'property_type',
       'cost_per_extra_person', 'has_system_columns', 'has_luxury_features',
       'infrastructure_score', 'facility_score', 'density_per_floor',
       'dir_east', 'dir_north', 'dir_south', 'dir_west', 'dir_نامشخص',
       'ad_year', 'building_age', 'effective_age', 'floor_material_rank',
       'deal_type', 'rent/sell', 'target'],
      dtype='object')


In [76]:
sell_df = sell_df.drop(['land_size', 'cost_per_extra_person', 'total_floors_count', 'has_heating_system', 'has_cooling_system' , 'has_warm_water_provider' , 'ad_year', 'building_age', 'rent/sell'], axis=1)

In [77]:
sell_df['location_combined'] = sell_df['city_slug'].astype(str) + '__' + sell_df['neighborhood_slug'].astype(str)

In [78]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
sell_df['location_encoded'] = le.fit_transform(sell_df['location_combined'])

In [73]:
sell_df = sell_df.drop(['city_slug', 'neighborhood_slug'], axis=1)

In [76]:
sell_df = sell_df.drop(['location_combined'], axis=1)

In [77]:

le = LabelEncoder()
sell_df['user_type_encoded'] = le.fit_transform(sell_df['user_type'])

In [78]:
print(sell_df['user_type_encoded'].value_counts())

user_type_encoded
2    422889
1    155700
0     18980
Name: count, dtype: int64


In [79]:
sell_df = sell_df.drop(['user_type'], axis=1)

In [83]:
material_rank = {
    'carpet': 0,
    'mosaic': 1,
    'ceramic': 2,
    'stone': 3,
    'laminate_parquet': 4,
    'wood_parquet': 5
}

In [85]:
sell_df['floor_material'] = sell_df['floor_material'].replace('floor_covering', np.nan)
sell_df['floor_material'] = sell_df['floor_material'].replace('unselect', np.nan)

In [86]:
sell_df['floor_material_rank'] = sell_df['floor_material'].map(material_rank)

In [88]:
sell_df['floor_material_rank'] = sell_df['floor_material_rank'].fillna(0).astype(int)

In [93]:
sell_df = sell_df.drop(['floor_material'], axis=1)

In [102]:

print(sell_df['property_type_encoded'].value_counts())

property_type_encoded
4    577828
1     15061
0      2725
2      1805
3       150
Name: count, dtype: int64


In [99]:
sell_df['property_type'] = sell_df['property_type'].replace('other', np.nan)

In [100]:
le = LabelEncoder()
sell_df['property_type_encoded'] = le.fit_transform(sell_df['property_type'])

In [103]:
sell_df = sell_df.drop(['property_type'], axis=1)

In [106]:
sell_df = sell_df[~sell_df['price_mode'].isin(['توافقی', 'مجانی'])]

In [108]:
sell_df = sell_df[(sell_df['price_value'].notna()) & (sell_df['price_value'] >= 1000000000)]

In [110]:
low = sell_df['price_value'].quantile(0.10)
high = sell_df['price_value'].quantile(0.90)

sell_df = sell_df[(sell_df['price_value'] >= low) & (sell_df['price_value'] <= high)]

In [115]:
sell_df = sell_df.drop(['price_mode'], axis=1)

In [116]:
sell_df.shape
sell_df.info

<bound method DataFrame.info of         Unnamed: 0     created_at_month   price_value  land_size    deed_type  \
1                1  2024-05-01 00:00:00  8.500000e+09        NaN          NaN   
4                4  2024-05-01 00:00:00  5.750000e+09        NaN  single_page   
7                7  2024-09-01 00:00:00  8.700000e+09        NaN          NaN   
9                9  2024-10-01 00:00:00  3.000000e+09        NaN          NaN   
10              10  2024-11-01 00:00:00  2.600000e+09        NaN        other   
...            ...                  ...           ...        ...          ...   
999990      999990  2024-09-01 00:00:00  4.000000e+09        NaN          NaN   
999991      999991  2024-12-01 00:00:00  3.600000e+09        NaN  single_page   
999992      999992  2024-11-01 00:00:00  4.350000e+09        NaN  single_page   
999995      999995  2024-07-01 00:00:00  7.470000e+09        NaN          NaN   
999997      999997  2024-11-01 00:00:00  3.200000e+09      200.0          NaN

In [129]:
sell_df.describe

<bound method NDFrame.describe of         Unnamed: 0     created_at_month   price_value  land_size floor  \
1                1  2024-05-01 00:00:00  8.500000e+09        NaN     3   
4                4  2024-05-01 00:00:00  5.750000e+09        NaN     4   
7                7  2024-09-01 00:00:00  8.700000e+09        NaN     4   
9                9  2024-10-01 00:00:00  3.000000e+09        NaN     3   
10              10  2024-11-01 00:00:00  2.600000e+09        NaN     4   
...            ...                  ...           ...        ...   ...   
999990      999990  2024-09-01 00:00:00  4.000000e+09        NaN   1.0   
999991      999991  2024-12-01 00:00:00  3.600000e+09        NaN   4.0   
999992      999992  2024-11-01 00:00:00  4.350000e+09        NaN   2.0   
999995      999995  2024-07-01 00:00:00  7.470000e+09        NaN   4.0   
999997      999997  2024-11-01 00:00:00  3.200000e+09      200.0   NaN   

        has_system_columns  has_luxury_features infrastructure_score  \
1    

In [121]:
sell_df['deed_type'] = sell_df['deed_type'].replace('other', np.nan)
sell_df['deed_type'] = sell_df['deed_type'].replace('unselect', np.nan)

In [122]:
le = LabelEncoder()
sell_df['deed_type_encoded'] = le.fit_transform(sell_df['deed_type'])

In [123]:
print(sell_df['deed_type_encoded'].value_counts())

deed_type_encoded
3    202266
1    156046
2     20330
0      4928
Name: count, dtype: int64


In [126]:

sell_df['has_business_deed'] = sell_df['has_business_deed'].replace(False, np.nan)

le = LabelEncoder()
sell_df['has_business_deed_encoded'] = le.fit_transform(sell_df['has_business_deed'])

In [127]:
print(sell_df['has_business_deed_encoded'].value_counts())

has_business_deed_encoded
1    375263
0      8307
Name: count, dtype: int64


In [128]:
sell_df = sell_df.drop(['has_business_deed', 'deed_type'], axis=1)

In [143]:
for col in sell_df.columns:
    print(f"{col}: {sell_df[col].dtype}")

Unnamed: 0: int64
created_at_month: object
price_value: float64
land_size: float64
floor: Int64
has_system_columns: int64
has_luxury_features: bool
infrastructure_score: Int64
facility_score: Int64
density_per_floor: Float64
dir_east: bool
dir_north: bool
dir_south: bool
dir_west: bool
dir_نامشخص: bool
effective_age: Int64
floor_material_rank: int64
deal_type: float64
target: float64
location_encoded: int64
user_type_encoded: int64
property_type_encoded: int64
deed_type_encoded: int64
has_business_deed_encoded: int64
ad_year: int32
adjusted_price: float64


In [135]:
sell_df['infrastructure_score'] = pd.to_numeric(sell_df['infrastructure_score'], errors='coerce').astype('Int64')
sell_df['effective_age'] = pd.to_numeric(sell_df['effective_age'], errors='coerce').astype('Int64')
sell_df['facility_score'] = pd.to_numeric(sell_df['facility_score'], errors='coerce').astype('Int64')

In [137]:
sell_df['floor'] = pd.to_numeric(sell_df['floor'], errors='coerce').astype('Int64')

In [140]:
sell_df['ad_year'] = pd.to_datetime(sell_df['created_at_month']).dt.year

In [142]:


# دیکشنری شاخص CPI با سال به عنوان کلید و مقدار CPI به عنوان مقدار
cpi_dict = {
    2020: 298.858,
    2021: 437.042,
    2022: 640.225,
    2023: 934.324,
    2024: 1000.0,
}

# سال پایه برای تعدیل (مثلا ۱۳۹۵)
base_year = 2024
base_cpi = cpi_dict[base_year]

# تابع برای تعدیل قیمت بر اساس CPI
def adjust_price(row):
    year = row['ad_year']
    price = row['price_value']
    if pd.isna(year) or pd.isna(price):
        return price  # اگر سال یا قیمت خالی بود، تغییر نده
    if year not in cpi_dict:
        return price  # اگر سال در دیکشنری نبود، تغییر نده
    adjusted_price = price * (base_cpi / cpi_dict[year])
    return adjusted_price

# اعمال تابع روی دیتافریم و ایجاد ستون جدید
sell_df['adjusted_price'] = sell_df.apply(adjust_price, axis=1)


In [144]:
sell_df = sell_df.drop(['created_at_month', 'price_value', 'ad_year'], axis=1)

In [145]:
features = sell_df.columns.drop('adjusted_price').tolist()


In [146]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [152]:
sell_df['log_price'] = np.log1p(sell_df['adjusted_price'])

In [151]:
features = sell_df.columns.drop('log_price','adjusted_price').tolist()

In [153]:
X = sell_df[features]
y = sell_df['log_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_pred_log = model.predict(X_test)

# تبدیل پیش‌بینی‌ها به مقیاس اصلی قیمت
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.2f}")
print(f"R2 score: {r2:.2f}")

MSE: 28541778156472733696.00
R2 score: -77313669012978139136.00
